In [4]:
#loading libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

from itertools import combinations 
import time
from sklearn.model_selection import train_test_split
import surprise
import ast
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
from scipy.spatial import distance_matrix


#from sklearn.metrics.pairwise import nan_euclidean_distances
#!pip install tensorflow
#import tensorflow as tf
#from sklearn.preprocessing import normalize
#from heapq import nlargest
#from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
"""
1. Hyperparameter tuning with cross validation


2. Run the model w/ the optimum parameters 
"""


ratings_v2=ratings.copy()
data=ratings_v2.rename(
    columns={'userId':'userID','movId':'itemID'}).sort_values(
    by=['userID','itemID']).drop(columns=['timestamp']).reset_index(drop=True)


#Split dataset in training and test
trainset, testset = train_test_split(data, test_size=0.025, random_state =123)


# A reader is still needed but only the rating_scale param is requiered.
reader = surprise.Reader(rating_scale=( data.rating.min(), data.rating.max() ))
# The columns must correspond to user id, item id and ratings (in that order).
trainset = surprise.Dataset.load_from_df(trainset, reader)


#Implement Grid Search
"""param_grid = {'n_factors': [2, 5, 10], 'n_epochs': [10, 15, 20],
              'biased': [True],
              'init_mean': [-1, 0, 1], 'init_std_dev': [0.01],
              'lr_bu': [0.005, 0.002], 'lr_bi': [0.005, 0.002],
              'lr_pu': [0.005, 0.002], 'lr_qi': [0.005, 0.002],
              'reg_bu': [0.05, 0.02, 0.01], 'reg_bi': [0.05, 0.02, 0.01],
              'reg_pu': [0.05, 0.02, 0.01], 'reg_qi': [0.05, 0.02, 0.01]
             }"""

param_grid = {'n_factors': [2,3,5,7,10], 'n_epochs': [10],
              'biased': [True],
              'init_mean': [0], 'init_std_dev': [0.01],
              'lr_bu': [0.05, 0.02, 0.01], 'lr_bi': [0.05, 0.02, 0.01],
              'lr_pu': [0.05, 0.02, 0.01], 'lr_qi': [0.05, 0.02, 0.01],
              'reg_bu': [0.1, 0.05, 0.01], 'reg_bi': [0.1, 0.05, 0.01],
              'reg_pu': [0.1, 0.05, 0.01], 'reg_qi': [0.1, 0.05, 0.01]
             }

algo=surprise.prediction_algorithms.matrix_factorization.SVD
gs=surprise.model_selection.search.GridSearchCV(algo, param_grid, measures=['mae', 'rmse', 'fcp'], cv=5,
                                                refit=False, return_train_measures=True, n_jobs=1,
                                                pre_dispatch='2*n_jobs', joblib_verbose=100)

display(gs.fit(trainset))
error=pd.DataFrame.from_dict(gs.cv_results)
for me in ['rmse', 'mae','fcp']:
    
    print('Error metric: ', me)
    print('best score       : ', gs.best_score[me])
    print('best parameters  : ', gs.best_params[me], '\n')
    

algo = gs.best_estimator["mae"]
algo.fit(trainset.build_full_trainset())
    
"""#fit the algorothm with the best parameters
algoOpt=gs.best_estimator["mae"]
modelOpt=surprise.model_selection.validation.cross_validate(algoOpt, data, measures=['mae', 'rmse', 'fcp'], cv=5,
                                                            return_train_measures=True, n_jobs=1,
                                                            pre_dispatch='2*n_jobs', verbose=True)

display(pd.DataFrame.from_dict(modelOpt))"""

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.9min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  5.7min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  8.5min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 11.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 13.9min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 16.6min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed: 19.3min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 22.0min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 24.8min remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 27.6min remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed: 30.4min remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  

In [ ]:
"""ttest = surprise.Reader(rating_scale=(1,5))
ttest = surprise.Dataset.load_from_df(testset, ttest)
ttest = ttest.build_testset()"""


algop = gs.best_estimator["mae"]
algop.fit(trainset.build_full_trainset())

predictions = algop.test(list(testset.itertuples(index=False, name=None)))
print('MAE on testset')
surprise.accuracy.mae(predictions, verbose=True)

predictions = algop.test( list(data[~data.index.isin(testset.index)].itertuples(index=False, name=None)) )
print('\nMAE on trainset')
surprise.accuracy.mae(predictions, verbose=True)



In [ ]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

In [ ]:
trainset=ratings_v1.rename(
    columns={'userId':'userID','movId':'itemID'}).sort_values(
    by=['userID','itemID']).drop(columns=['timestamp']).reset_index(drop=True)

# A reader is still needed but only the rating_scale param is requiered.
reader = surprise.Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = surprise.Dataset.load_from_df(trainset, reader)



algo=surprise.prediction_algorithms.matrix_factorization.SVD


gs=surprise.model_selection.search.GridSearchCV(algo, param_grid, measures=['rmse', 'mae'], cv=5,
                                                refit=True, return_train_measures=True, n_jobs=1,
                                                pre_dispatch='2*n_jobs', joblib_verbose=30)

gs.fit(data)

# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

In [ ]:
algo1=gs.best_estimator["mae"]
algo1.fit(data.build_full_trainset())

In [2]:
#load datasets

users=pd.read_csv(r"C:\Users\GonçaloVieira\OneDrive - Metyis\Bureaublad\Lap\ML\datasets\RS\medium\users.dat",
                  sep="::",header=None,names=['userId','gender','age','job','zipcode'])
ratings=pd.read_csv(r"C:\Users\GonçaloVieira\OneDrive - Metyis\Bureaublad\Lap\ML\datasets\RS\medium\ratings.dat",
                    sep="::",header=None,names=['userId','movieId','rating','timestamp'])
movies=pd.read_csv(r"C:\Users\GonçaloVieira\OneDrive - Metyis\Bureaublad\Lap\ML\datasets\RS\medium\movies.dat",
                   sep="::",header=None,names=['movieId','title','genres'],encoding='latin-1')
movDetails=pd.read_csv(r"C:\Users\GonçaloVieira\OneDrive - Metyis\Bureaublad\Lap\ML\datasets\RS\webScrapingTMDB\results.csv")

C:\Users\GonçaloVieira\AppData\Local\Temp\ipykernel_29108\3844444380.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users=pd.read_csv(r"C:\Users\GonçaloVieira\OneDrive - Metyis\Bureaublad\Lap\ML\datasets\RS\medium\users.dat",
C:\Users\GonçaloVieira\AppData\Local\Temp\ipykernel_29108\3844444380.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings=pd.read_csv(r"C:\Users\GonçaloVieira\OneDrive - Metyis\Bureaublad\Lap\ML\datasets\RS\medium\ratings.dat",
C:\Users\GonçaloVieira\AppData\Local\Temp\ipykernel_29108\3844444380.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' en

In [ ]:
#Datasets Structure

print('DATASET users\n')
print(users.info())
display(users)

print('\n','*'*50,'\n')
print('DATASET movies\n')
print(movies.info())
display(movies)

print('\n','*'*50,'\n')
print('DATASET ratings\n')
print(ratings.info())
display(ratings)

In [ ]:
"""
EDA (Exploratory Data Analysis)

- how many users? how many movies?
- how many rated movies? how many tagged movies?
- # rated movies per user
- # users per rated movies
- ratings: median, mean, max, min
- unique values of each column, relative and absolute frequency
- standard deviation, top 5 & bottom-5 regarding the frequency

"""
print('EDA on users dataset\n')
print('num of users            : ',users.userId.nunique())
print('userId indexes          : ',users.userId.min() ,'-',users.userId.max())
print('users age               : ',users.age.min() ,'-',users.age.max())
print('users gender            : ',users.gender.unique())
print('num of user jobs        : ',users.job.nunique())
print('num of user location    : ',users.zipcode.nunique(),'\n'*2,'*'*70)

print('\nEDA on movies dataset\n')
print('num of movies           : ',movies.movieId.nunique())
print('movieID indexes         : ',movies.movieId.min() ,'-',movies.movieId.max())
print('movie release year      : ',
      movies.title.str.slice(-5,-1).astype(int).min() ,'-',movies.title.str.slice(-5,-1).astype(int).max(),
     '\n'*2,'*'*70)

print('\nEDA on ratings dataset\n')
print('num of users that rated : ',ratings.userId.nunique())
print('num of rated movies     : ',ratings.movieId.nunique())
print('rating values           : ',np.sort(ratings.rating.unique()) )

aux=ratings.groupby(by=['userId','movieId']).rating.count().reset_index().sort_values('rating',ascending=False)
print('Number of users that rated the same movie multiple times: ',aux[aux.rating>1].userId.nunique(),'users')

**Transformations**

+ **Movies Dataset (movieId, title, genres)**
    
    > Create a variable indicating the number of ratings and order the dataset in
    descending order considering this variable.
    
    > Create a similar feature to movieId named movId. This variable will be used
    in the model due to convenience
    
+ **Movies Details Dataset**

    > Through hot encoding, add new columns with all movie genres
    
    
+ **Ratings Dataset (userId, movieId, rating, timestamp )**

    > Add movId feature to this dataset
    
    > Format timestamp feature
    
    > Format userId feature
    
+ **Users Dataset (userId, gender, age, job, zipcode )**

    > Format userId feature
    
    > Add feature especifing the type of job a certain user has, jobT

In [3]:
aux=ratings.groupby('movieId').userId.count().reset_index()
aux.rename(columns={'userId':'nRatings'},inplace=True)
movies=movies.merge(aux, how='left',on='movieId')
movies.nRatings.fillna(0,inplace=True)
movies.nRatings=movies.nRatings.astype(int)
movies.sort_values(by=['nRatings'],ascending=[False],inplace=True)
movies.reset_index(inplace=True)
movies['movId']=movies.index
movies=movies[['movieId','movId','title','nRatings','genres']]
#movies[['movieId','movId','title','genres']].to_csv(r"C:\Users\GonçaloVieira\OneDrive - Metyis\Bureaublad\Lap\ML\datasets\RS\webScrapingTMDB\moviesMed.csv")



movDetails.drop(columns=['Unnamed: 0'], inplace=True)
movDetails.crew=movDetails.crew.apply(ast.literal_eval)
movDetails.cast=movDetails.cast.apply(ast.literal_eval)
movDetails.tags=movDetails.tags.apply(lambda x: x[1:-1].replace("'","").split(','))
movDetails=movDetails.merge(movies[['movId','title']], how='left', left_on='titleM',right_on='title')
movDetails.drop(columns=['title_y'], inplace=True)
movDetails.rename(columns={'title_x':'title'}, inplace=True)

aux=movies['genres'].str.split('|',expand=True)
aux['title']=movies.title
aux=pd.melt(aux,id_vars='title',value_name='genre')
aux=aux[['title','genre']]
aux.dropna(inplace=True)
aux['val']=1
aux=aux.set_index(['title','genre']).val.unstack().fillna(0)
display
movDetails=movDetails.merge(aux, how='left', left_on='titleM',right_on=aux.index)



ratings=ratings.merge(movies[['movieId','movId']],how='left', on='movieId')
ratings=ratings[['userId','movId','rating','timestamp']]
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'],unit='s')
ratings.userId=ratings.userId-1
ratings.sort_values(by=['userId','movId'], inplace=True)
ratings.reset_index(drop=True, inplace=True)



users.userId=users.userId-1
aux={
    0:  "other or not specified",
    1:  "academic/educator",
    2:  "artist",
    3:  "clerical/admin",
    4:  "college/grad student",
    5:  "customer service",
    6:  "doctor/health care",
    7:  "executive/managerial",
    8:  "farmer",
    9:  "homemaker",
    10:  "K-12 student",
    11:  "lawyer",
    12:  "programmer",
    13:  "retired",
    14:  "sales/marketing",
    15:  "scientist",
    16:  "self-employed",
    17:  "technician/engineer",
    18:  "tradesman/craftsman",
    19:  "unemployed",
    20:  "writer",
}
users=users.merge(pd.DataFrame(aux.items(),columns=['job','jobT']), how='left', on='job')
aux={
    1:  "Under 18",
    18:  "18-24",
    25:  "25-34",
    35:  "35-44",
    45:  "45-49",
    50:  "50-55",
    56:  "56+"
}
users=users.merge(pd.DataFrame(aux.items(),columns=['age','ageT']), how='left', on='age')
users=users[['userId', 'gender','age','ageT','job','jobT','zipcode']]

In [ ]:
"""
EDA (Exploratory Data Analysis)
Double Check

    - Number of distinct movies matches movie lowest index and movie greatest index
    - Number of distinct users matches user lowest index and user greatest index
    
"""
print('Double Checks\n')
print('dc #1:\n','# unique movies: ',movies.movieId.nunique(),'// indexes:',movies.movId.min(),'-',movies.movId.max())
print('dc 21:\n','# unique users : ',ratings.userId.nunique(),' // indexes:',ratings.userId.min(),'-',ratings.userId.max())


print('Stats after transformations:')
print('\nNum of ratings per user')
display(ratings.groupby('userId').movId.count().describe().round())

print('\nNum of ratings per movie')
display(ratings.groupby('movId').userId.count().describe().round())

print('\nratings - summary stats')
display(ratings.rating.describe().round(2))


"""print('\nMost seen movies per category')
display(
    ratings.merge(movies.loc[:,['movId']+list(movies.columns[-18:])] ,how='left',
             on='movId').iloc[:,4:].sum(axis=0).sort_values(ascending=False)/ratings.shape[0]*100
)

print('\nMost seen movies per pairs of categories')
aux=ratings_v1.merge(movies.loc[:,['movId']+list(movies.columns[-18:])], how='left',
                  on='movId')
aux1=[]
for p in combinations(movies.drop(columns=movies.columns[:-18]).columns,2):
    aux1.append([p[0],p[1],aux[(aux[p[0]]==1) & (aux[p[1]]==1)].shape[0]])

display(pd.DataFrame(aux1,columns=['genre1','genre2','freq']).sort_values('freq',ascending=False).head(10))"""

print('\nMost popular movies')
display( movies[['movId','title','nRatings']].sort_values(by='nRatings',ascending=False).head() )

In [ ]:
aux=ratings[['userId','movId','rating']].copy()

aux=aux.groupby('movId').userId.count().reset_index()
aux.rename(columns={'userId':'nRatings'},inplace=True)

fig, axs = plt.subplots()
sns.set(style="darkgrid")
sns.histplot(data=aux.nRatings, bins=30, kde=False)
axs.set(xlabel='Number of ratings', ylabel='Number of movies')
plt.show()

In [ ]:
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules

aux=ratings[ratings.rating>=4][['userId','movId']]
aux=aux.merge(movies[['movId','title']], on='movId', how='left')
aux['hotEncode']=True
aux.set_index(['userId','title'])['hotEncode'].unstack().fillna(False)

assocRules=association_rules(
    fpgrowth(aux.set_index(['userId','title'])['hotEncode'].unstack().fillna(False),
         min_support=0.10,
         use_colnames=True),
    metric="confidence",
    min_threshold=0.70)
assocRules.antecedents=assocRules.antecedents.astype(str).str.slice(start=11, stop=-2)
assocRules.consequents=assocRules.consequents.astype(str).str.slice(start=11, stop=-2)

assocRules.to_csv(r"C:\Users\GonçaloVieira\OneDrive - Metyis\Bureaublad\Lap\ML\datasets\RS\EDA\AssociationRules\assocRules.csv")
assocRules

In [ ]:
"""
- Realizar um cluster considerando os generos de filmes
Para cada user determinar a porçao de ratings (>=4) para cada genero de filme

    > there are 2 users whose ratings are all less than 4
    set(users.userId)-set(clUsers.index)
    

"""

clUsers=ratings.merge(movDetails.iloc[:,14:], on='movId', how='left').drop(columns=['timestamp'])
clUsers=clUsers[clUsers.rating>=4].drop(columns=['movId', 'rating'])
clUsers=clUsers.groupby(by=['userId']).sum()
clUsers=clUsers/np.sum(np.array(clUsers), axis=1, keepdims=True)
display(clUsers)

def kmeansModel(data,k):
    """
    inertia = sum of squared distances between each point and the closest centroid
    w_in    = sum of intra cluster distances
    w_out   = sum of inter cluster distances

    N_in    = num of intra cluster edges
    N_out   = num of inter cluster edges
    
    silhouette_score can be calculated in the following way:
    > silhouette_score(model_1n.iloc[:,2:], kmeans.labels_, metric='euclidean')    

    """

    kmeans = KMeans(n_clusters=k, init='k-means++', n_init=10).fit(np.array(data))
    #print(kmeans.cluster_centers_)
    #print('inertia: ',kmeans.inertia_)
    aux=data.copy()
    aux['label']=kmeans.labels_

    inertia=0
    w_in=0
    w_out=0
    N_in=0
    N_out=0

    for j in range(k):

        inertia = inertia + np.sum((np.array(aux[aux.label==j].iloc[:,:-1])-kmeans.cluster_centers_[j])**2)
        w_in=w_in+np.sum(distance_matrix(aux[aux.label==j].iloc[:,:-1],aux[aux.label==j].iloc[:,:-1],p=2))
        w_out=w_out+np.sum(distance_matrix(aux[aux.label==j].iloc[:,:-1],aux[aux.label!=j].iloc[:,:-1],p=2))

        N_in=N_in+( aux[aux.label==j].shape[0] * (aux[aux.label==j].shape[0]-1) )
        N_out=N_out+( aux[aux.label==j].shape[0] * aux[aux.label!=j].shape[0] )

    w_in=w_in/2
    w_out=w_out/2
    N_in=int(N_in/2)
    N_out=int(N_out/2)

    #print('inertia : ', inertia)
    #print('w_in    : ', w_in)
    #print('w_out   : ', w_out)
    #print('N_in    : ', N_in)
    #print('N_out   : ', N_out)
    #print('BetaCV  : ', (w_in/N_in)/(w_out/N_out))
    
    
    """# Compute Gap Statistic
    feats=list(aux.columns[:-1])
    w_inR_values=[]
    for t in range(100):
        R=[]
        for c in feats:
            R.append(np.random.uniform(aux[c].min(),aux[c].max(),data.shape[0]))
        R=pd.DataFrame(np.array(R).T,columns=feats)
        R['label']=KMeans(n_clusters=k, init='k-means++', n_init=10).fit(R).labels_
        
        w_inR=0
        for j in range(k):
            w_inR=w_inR+np.sum(distance_matrix(R[R.label==j].iloc[:,:-1],R[R.label==j].iloc[:,:-1],p=2))
        w_inR_values.append(w_inR)
    mean_w_in=np.mean(np.log2(w_inR_values))
    std_w_in =np.std(np.log2(w_inR_values))
    gap=mean_w_in-np.log2(w_in)"""
            
            
            
            
    return (inertia,w_in,w_out,N_in,N_out,(w_in/N_in)/(w_out/N_out),
            np.mean(silhouette_samples(data, kmeans.labels_, metric='euclidean')),
            silhouette_samples(data, kmeans.labels_, metric='euclidean'),
            #gap, std_w_in, 
            aux)


# 2. K-Means

nclusters=range(2,19)
w_in=[]
betaCV=[]
silhouette_score=[]
gap=[]
gap_std=[]

for j in nclusters:
    
    model=kmeansModel(clUsers,j)
    w_in.append(model[1])
    betaCV.append(model[5])
    silhouette_score.append(model[6])
    #gap.append(model[8])
    #gap_std.append(model[9])

fig,axs=plt.subplots(1,3,figsize=(30, 10))
axs[0].plot(nclusters, w_in, '-o')
axs[0].set_title('Intra Cluster Distance')
axs[1].plot(nclusters, betaCV, '-o')
axs[1].set_title('BetaCV')
axs[2].plot(nclusters, silhouette_score, '-o')
axs[2].set_title('Silhouette Score')
"""axs[1,1].errorbar(nclusters, gap, gap_std, 
             linestyle='-', marker='o',
            ecolor='red',capsize=10)
axs[1,1].set_title('Gap Statistic')"""
plt.show()

In [ ]:
"""
Results from the clustering
k=5, 7 or 8


"""

kmeans = KMeans(n_clusters=8, init='k-means++', n_init=10, random_state=123).fit(np.array(clUsers))
clUsers['label']=kmeans.labels_
clUsers

In [ ]:
for lab in clUsers.label.unique():
    
    print('cluster #',lab)
    aux=clUsers[clUsers.label==lab]
    print('cluster size: ', aux.shape[0])
    print(aux.iloc[:,:-1].sum(axis=0).sort_values(ascending=False).head(3),'\n')

In [ ]:
np.array([[1,2],[3,4],[5,6]]).sum(axis=0)


**Model**

**A. Collab Filtering**
    
    1. Hold Out : Split data in 3 datasets
        - training     95%
        - validation   2.5%
        - test         2.5%
    
    2. Algorithm:
        2.1 Obtain similarity matrix among users
            [* To compare 2 users, they must have a minimum number of items in common
            ( ncm is used as a parameter ) pd.DataFrame.corr(method='pearson', min_periods=ncm)]
            
        2.2 Compute the neighborhood of each user
        
            2.2.1 For each target user his neighborhood must satisfy the following conditions:
                ** Each neighbor must be similar enough to the target user, accessed by min_sim
                ** The whole neighborhood must have exactly nnei neighbors that respect the 
                previous condition, excluding the target user
                
                For users in the such conditions the Collab Filtering algorithm will predict 
                recommendations.
                The other users will have recommendations based on Naive Bayes CF.
                
                It is not necessary that the whole neighborhood contributes to the prediction
                of a movie. It can happen, a recommendation for a certain user came from only 1 of 
                his neighbors. Note also that a user may not have a neighborhood but can be 
                in some neighborhood. Therefore, even though CF may not be able to recommend items 
                to certain user, the user's ratings can be used to recommend items to other users.
                
                
            
        2.3 Compute items - potencial recommendations, for each user
        2.4 Predict ratings for the potencial recommendation items
         
    3. Access avaliation metrics on all predicted movies and top-5 recommendations per user
        - Hit Rate (%)
        'Hit Rate (%)': np.round(ratingsPredValid.shape[0]/dvaldCV[dvaldCV.userId.isin(cfUsers)].shape[0]*100,1)
        
        - Precision (RMSE, MAE, NRMSE, NMAE)
        - Coverage + Diversity
        - Novelty + Serendipity
        
        
**To do**

    - rmatrix pode conter 0s por causa de nao haver rating, ie, os users nao viram
    um determinado filme OU entao por que resulta de r_uj:=r_uj-media_u
    ESTE problema esta em >>> rmatrix_n_neiu_predm!=0
    MESMO problema em mov_pred (ja resolvido!!?)

In [ ]:
def dataSplit(ratings,ptest):
    
    """
    Split ratings dataset in 3 smaller datasets:
        - Training
        - Validation
        - Test
    Note that
        - validation dataset will be the same size as test dataset
        - Hold Out or Cross validation 
        - validation dataset is within training dataset   """
    
    dtrain,dtest=train_test_split(ratings,
                       test_size=ptest, random_state=123, shuffle=True)
    dtest.sort_values(by=['userId','movId'],inplace=True)
    dtrain.sort_values(by=['userId','movId'],inplace=True)
    
    cv=np.repeat( range(1,int(ptest**(-1))+1), np.floor(dtrain.shape[0]*ptest) )
    cv=np.concatenate((cv,np.random.choice(range(1,int(ptest**(-1))+1), size=dtrain.shape[0]-cv.shape[0], replace=False)))
    np.random.shuffle(cv)
    dtrain['cvSplit']=cv
    
    aux=ratings.groupby('movId').userId.count().reset_index().merge(
        dtrain.groupby('movId').userId.count().reset_index(), how='left', on='movId')
    print('Which movie contributes the most in the test dataset? or\nthe highest % of ratings used from a certain movie in the test dataset is :',
          np.round(100-(aux.userId_y/aux.userId_x*100).min(),1),'%')
    aux=ratings.groupby('userId').movId.count().reset_index().merge(
        dtrain.groupby('userId').movId.count().reset_index(), how='left', on='userId')
    print('Which user contributes the most in the test dataset? or\nthe highest % of ratings used from a certain user in the test dataset is  :',
          np.round(100-(aux.movId_y/aux.movId_x*100).min(),1),'%\n')
    
    return dtrain,dtest


def simMatrix(ratings, mncmr):
    
    start=time.time()
    #compute similarity matrix using surprise package
    aux=ratings.copy()
    aux['userRat'] = list(zip(aux.userId, aux.rating))
    simUsers={}

    for m in np.sort(aux.movId.unique()):
        simUsers[m]=list(aux[aux.movId==m].userRat)
        
    simUsers=surprise.similarities.msd(aux.userId.nunique(), simUsers, min_support=mncmr)
    np.fill_diagonal(simUsers, 0)
        
    print('Similarity matrix computation time (secs): ', np.round(time.time()-start,1))
    return simUsers
    
    
def neighborhood(sim, nnei, simMin):
    
    """
    2.2 Compute neighborhoods & exclude non correlated neighbors.
    Compute users where CF will be applied. """
    nu=sim.shape[0]
    start=time.time()
    viz=[[np.sort(np.argpartition(sim[u], -nnei)[-nnei:]) 
         ,sim[u][np.sort(np.argpartition(sim[u], -nnei)[-nnei:])]]
                  
         for u in range(nu)]
    
    #exclude non correlated neighbors
    viz=[viz[u][0][np.where(viz[u][1]>=simMin)]
         for u in range(nu)]
    
    cfUsers=np.where(np.array([len(v) for v in viz])>=nnei)[0]
    ncfUsers=list(set(range(nu))-set(cfUsers))
    
    print('Neighborhood computation time (secs)     : ',np.round(time.time()-start,1))
    
    return viz, cfUsers, ncfUsers
    
def itemsToPredict(rmatrix, viz, cfUsers):
    
    start=time.time()
    movPred=[np.sort(list(set(np.where((~np.isnan(rmatrix[viz[u],:])).sum(axis=0)!=0)[-1]) -
                           set(np.where(~np.isnan(rmatrix[u,:]))[-1]) 
                          ))
              for u in cfUsers]

    print('Movies to predict computation time (secs): ',np.round(time.time()-start,1))
    
    return movPred


def itemsPredictions(rmatrixMC, cfUsers, movPred, viz, sim, rmatrix):
    
    start=time.time()
    pred=[]

    rmatrixMC0=rmatrixMC.copy()
    rmatrixMC0[np.isnan(rmatrixMC0)] = 0
    for u,v in zip(cfUsers, range(len(movPred))):

        rmatrixMC0_NeiuPredm=rmatrixMC0[viz[u]][:,movPred[v]]
        simUNeiu=sim[u,viz[u]]

        predU=np.matmul(simUNeiu,
                        rmatrixMC0_NeiuPredm)/np.matmul(np.abs(simUNeiu),
                                                        ~np.isnan(rmatrix[viz[u]][:,movPred[v]]))
        
        pred.append(predU)
        
    pred=[p+np.nanmean(rmatrix[u]) for p,u in zip(pred,cfUsers)]
    print('Prediction computation time (secs)       : ', np.round(time.time()-start,1),'\n')
    
    return pred

def collabFiltering(sim, nnei, simMin, rmatrix, rmatrixMC):          

    # 2.2 Compute neighborhoods and cfUsers
    viz, cfUsers, ncfUsers = neighborhood(sim, nnei, simMin)

    # 2.3 Compute movies to predict
    movPred=itemsToPredict(rmatrix, viz, cfUsers)

    # 2.4 Predict ratings and get top nri recommendations
    pred =itemsPredictions(rmatrixMC, cfUsers, movPred, viz, sim, rmatrix)
    
    return pd.DataFrame({'userId':np.repeat(cfUsers,[len(mo) for mo in movPred]),
                         'movId':np.hstack(movPred), 'ratingPred':np.hstack(pred)}), cfUsers, ncfUsers
    
#ola=simMatrix(rmatrixMC, 40)

In [ ]:
def rs_cf(ratings, ptest, crossValid):
    
    """
    Parameters:
    
    crossvalid=True or False          # whether or not to perform Cross Validation
    mncmr=37                          # number of common items (movies)
    simMin=0.3                        # minimum similarity between users to be in a neighborhood
    nnei=45                           # number of neighborhoods 
    
    """
    
    #Split original dataset in training and test
    dtrain, dtest = dataSplit(ratings,ptest)
    
    if crossValid==True:
        
        erro={}
        for mncmr in np.arange(45,50,5):
            for nnei in np.arange(40,50,5):
                for simMin in np.arange(0.5,0.6,0.1):
                    
                    NRMSE=[]
                    NMAE=[]
                    for i in np.sort(dtrain.cvSplit.unique()):

                        dvaldCV=dtrain[dtrain.cvSplit==i]
                        dtrainCV=dtrain[dtrain.cvSplit!=i]

                        # 2.1 Compute rating matrix and similarity matrix
                        rmatrix   = np.array(dtrainCV.set_index(['userId','movId']).rating.unstack())
                        rmatrixMC = rmatrix-np.nanmean(rmatrix, axis=1, keepdims=True)
                        sim=simMatrix(dtrainCV, mncmr)            

                        # 2.234 Collaborative Filtering
                        pred, cfUsers, ncfUsers =collabFiltering(sim, nnei, simMin, rmatrix, rmatrixMC)

                        # Access the prediction quality on the validation set
                        predVald=pred.merge(dvaldCV[['userId','movId','rating']], how='inner', on=['userId','movId'])
                        predVald.ratingPred=np.where(predVald.ratingPred>5,5,
                                                     np.where(predVald.ratingPred<1,1,predVald.ratingPred))

                        erroi={'% of users w/ ratings': np.round(100*len(cfUsers)/(len(cfUsers)+len(ncfUsers)),1),
                               'Normalized RMSE (NRMSE %)': np.round(
                                   100*np.sqrt(np.sum((predVald.ratingPred-predVald.rating)**2)/predVald.shape[0])/5,
                                   1),
                               'Normalized MAE (NMAE %)': np.round(
                                   100*np.sqrt(np.sum(np.abs(predVald.ratingPred-predVald.rating))/predVald.shape[0])/5,
                                   1)
                              }
                        NRMSE.append(erroi['Normalized RMSE (NRMSE %)'])
                        NMAE.append(erroi['Normalized MAE (NMAE %)'])

                    erroP={'Normalized RMSE (NRMSE %)': np.mean(NRMSE),
                           'Normalized MAE (NMAE %)': np.mean(NMAE)}
                    erro[(mncmr, nnei, simMin)]=erroP
        
        return dtrain, dtest, erro
    
    
    
    else:
        
        #Split training dataset in training and validation
        dvald=dtrain[dtrain.cvSplit==np.random.choice(dtrain.cvSplit.unique(), size=1, replace=False)[0] ]
        dtrain=dtrain[dtrain.cvSplit!=dvald.cvSplit.unique()[0]]
        
        # 2.1 Compute rating matrix
        rmatrix   = np.array(dtrain.set_index(['userId','movId']).rating.unstack())
        rmatrixMC = rmatrix-np.nanmean(rmatrix, axis=1, keepdims=True)
        
        erro={}
        for mncmr in np.arange(45,50,5):
            for nnei in np.arange(40,50,5):
                for simMin in np.arange(0.5,0.6,0.1):
                    
                    # 2.1 Compute similarity matrix
                    sim=simMatrix(dtrain, mncmr)            

                    # 2.234 Collaborative Filtering
                    pred, cfUsers, ncfUsers =collabFiltering(sim, nnei, simMin, rmatrix, rmatrixMC)

                    # Access the prediction quality on the validation set
                    hitRate=dvald[dvald.rating>=4].merge(
                        pred[pred.ratingPred>=4.5],
                        how='inner', on=['userId','movId']).sort_values(by=['userId','movId'])
                    
                    predVald=pred.merge(dvald[['userId','movId','rating']], how='inner', on=['userId','movId'])
                    
                    predVald.ratingPred=np.where(predVald.ratingPred>5,5,
                                                 np.where(predVald.ratingPred<1,1,predVald.ratingPred))

                    erroP={'% of users w/ ratings': np.round(100*len(cfUsers)/(len(cfUsers)+len(ncfUsers)),1),
                           'Normalized RMSE (NRMSE %)': np.round(
                               100*np.sqrt(np.sum((predVald.ratingPred-predVald.rating)**2)/predVald.shape[0])/5,
                               1),
                           'Normalized MAE (NMAE %)': np.round(
                               100*np.sqrt(np.sum(np.abs(predVald.ratingPred-predVald.rating))/predVald.shape[0])/5,
                               1),
                           'Hit Rate (%)': np.round(100*hitRate.userId.nunique()/len(cfUsers))
                          }
                    erro[(mncmr, nnei, simMin)]=erroP

        return dtrain, dvald, dtest, erro



In [ ]:
dtrain, dvald, dtest, erro = rs_cf(ratings_v1, 0.025, False)

In [ ]:
pred.ratingPred=np.where(pred.ratingPred>5,5,pred.ratingPred)
pred.merge(dtest, how='inner',on=['userId','movId']).sort_values(['ratingPred','rating'],ascending=[False,True]).head(50)

In [ ]:
ratingsPred_t=pred.merge(dtest[['userId','movId','rating']],how='inner',on=['userId','movId'])
ratingsPred_t.ratingPred=np.where(ratingsPred_t.ratingPred>5,5,
                                     np.where(ratingsPred_t.ratingPred<1,1,ratingsPred_t.ratingPred))
print('Normalized RMSE (NRMSE):\n',
      np.round(
          100*np.sqrt(np.sum((ratingsPred_t.ratingPred-ratingsPred_t.rating)**2)/ratingsPred_t.shape[0])/5 , 1) )
print('Normalized MAE (NMAE):\n',
      np.round(
          100*np.sqrt(np.sum(np.abs(ratingsPred_t.ratingPred-ratingsPred_t.rating))/ratingsPred_t.shape[0])/5, 1) )
print('Hit Rate:\n',
      np.round(ratingsPred_t.shape[0]/dtest[dtest.userId.isin(usersdiv['cf_users'])].shape[0]*100,1))

In [ ]:
"""

NAIVE BAYES Algorithm
for cb_users



ratings_v2=ratings.copy()
ratings_v2=ratings_v2.merge(movies[['movieId','movId']],how='left',
                            on='movieId')
ratings_v2=ratings_v2[['userId','movId','rating','timestamp']]
ratings_v2['timestamp'] = pd.to_datetime(ratings_v2['timestamp'],unit='s')
ratings_v2.userId=ratings_v2.userId-1
ratings_v2=ratings_v2[~ratings_v2.index.isin(dtest.index+dvald.index)]


#Split in training and test
ptest=2.5/100
aux=ratings_v2[ratings_v2.userId.isin(usersdiv['ncf_users'])]
dtest=aux.sample(n=int(ptest*aux.shape[0]),replace=False,random_state=123)
dtest.sort_index(inplace=True)

#Split in training and validation
dtrain=ratings_v2[~(ratings_v2.index.isin(dtest.index))]
pvald=2.5/100
dvald=dtrain[dtrain.userId.isin(usersdiv['ncf_users'])].sample(n=int(pvald*aux.shape[0]), replace=False,random_state=123)
dvald.sort_index(inplace=True)
dtrain=dtrain[~(dtrain.index.isin(dvald.index))]

aux=ratings_v2[ratings_v2.userId.isin(usersdiv['ncf_users'])].groupby('movId').userId.count().reset_index().merge(
    dtrain[dtrain.userId.isin(usersdiv['ncf_users'])].groupby('movId').userId.count().reset_index(), how='left', on='movId')
print('Which movie contributes the most in the test dataset? or\nthe highest % of ratings used from a certain movie in the test dataset is:',
      np.round(100-(aux.userId_y/aux.userId_x*100).min(),1),'%')
aux=ratings_v2[ratings_v2.userId.isin(usersdiv['ncf_users'])].groupby('userId').movId.count().reset_index().merge(
    dtrain[dtrain.userId.isin(usersdiv['ncf_users'])].groupby('userId').movId.count().reset_index(), how='left', on='userId')
print('Which user contributes the most in the test dataset? or\nthe highest % of ratings used from a certain user in the test dataset is:',
      np.round(100-(aux.movId_y/aux.movId_x*100).min(),1),'%')

nbP1=dtrain.copy()
nbP1['count']=1
nbP1=nbP1.groupby(by=['movId','rating'])['count'].sum().reset_index().sort_values(by=['movId','rating'])
nbP1=nbP1.pivot(index='movId', columns='rating', values='count').fillna(0)

#movies to predict
movies_pred=[]
movies_seen=[]
mov=set(dtrain.movId.unique())
for u in usersdiv['ncf_users']:
    movies_pred.append(list(mov-set(dtrain[dtrain.userId==u].movId.unique())))
    movies_seen.append(dtrain[dtrain.userId==u].movId.unique())
    
#predictions
alpha=1
l=5
nbP1=(nbP1+alpha).div(nbP1.sum(axis=1)+l*alpha,axis=0)
nbP1a=np.array(nbP1)

ratMov=dtrain.sort_values(by=['movId','rating'])
ratMov['ratMovId']=(ratMov.rating.astype(str)+ratMov.movId.astype(str))#.astype(int)
ratMov=ratMov.ratMovId.unique()
nbP2=pd.DataFrame(np.zeros( (ratMov.shape[0],ratMov.shape[0]) ),columns=ratMov)
nbP2.index=ratMov
start=time.time()
for rm in ratMov:
    
    m=int(str(rm)[1:])
    r=int(str(rm)[0])
    
    p2=dtrain[(dtrain.movId==m) & (dtrain.rating==r)][['movId','rating','userId']].sort_values(['rating','userId'])
    p2=dtrain[dtrain.userId.isin(p2.userId)].groupby(by=['movId','rating'])[['userId']].count().reset_index()
    p2.rename(columns={'userId':'nUsers'},inplace=True)
    p2['ratMovId']=(p2.rating.astype(str)+p2.movId.astype(str))
    
    nbP2.loc[rm,list(p2['ratMovId'])]=p2.nUsers.to_numpy()

nbP2=(nbP2+alpha).div(np.diag(np.array(nbP2))+l*alpha,axis=0)
display(nbP2)
end=time.time()
print('\nNaive Bayes DataFrame computation time (secs):',np.round(end-start,1))

preds=[]
for u, ms, mp in zip(usersdiv['ncf_users'], movies_seen, movies_pred):
    p1=nbP1[nbP1.index.isin(mp)]
    
    display(p1)
        
        """


In [ ]:
from surprise import KNNWithMeans, Reader, Dataset
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split


# Creation of the dataframe. Column names are irrelevant.

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(ratings_v1[["userId", "movId", "rating"]], reader)

trainset, testset = train_test_split(data, test_size=0.05)


# We'll use the famous SVD algorithm.
sim_options = {
    "name": "pearson",
    "user_based": True,  # compute  similarities between users
    'min_support':35
}
algo = KNNWithMeans(min_k=30,sim_options=sim_options)

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=["RMSE", "MAE","FCP"], cv=5, verbose=True)

In [ ]:
"""

Greed Search

"""

#nnei=4                            # number of neighborhoods (user is in its own neighborhood)
#min_nei=3                         # minimum number of neighborhoods
#ncm=5                             # number of common items (movies)
#min_sim=0.6                       # minimum similarity between users

gsearch=[]

for min_nei in [35,40,45]:
    for ncm in [35,37,40,43,45]:
        for min_sim in [0.3]:
            nnei=min_nei+1


            #Split in training and test
            ptest=2.5/100
            dtest=ratings_v1.sample(n=int(ptest*ratings_v1.shape[0]), replace=False,random_state=123456)
            dtest.sort_index(inplace=True)

            #Split in training and validation
            dtrain=ratings_v1[~(ratings_v1.index.isin(dtest.index))]
            pvald=2.5/100
            dvald=dtrain.sample(n=int(pvald*ratings_v1.shape[0]), replace=False,random_state=1234)
            dvald.sort_index(inplace=True)
            dtrain=dtrain[~(dtrain.index.isin(dvald.index))]

            aux=ratings_v1.groupby('movId').userId.count().reset_index().merge(
                dtrain.groupby('movId').userId.count().reset_index(), how='left', on='movId')
            #print('Which movie contributes the most in the test dataset? or\nthe highest % of ratings used from a certain movie in the test dataset is:',
            #      np.round(100-(aux.userId_y/aux.userId_x*100).min(),1),'%')
            aux=ratings_v1.groupby('userId').movId.count().reset_index().merge(
                dtrain.groupby('userId').movId.count().reset_index(), how='left', on='userId')
            #print('Which user contributes the most in the test dataset? or\nthe highest % of ratings used from a certain user in the test dataset is:',
            #      np.round(100-(aux.movId_y/aux.movId_x*100).min(),1),'%')


            #parameters

            nu=ratings_v1.userId.nunique()    # number of users
            nm=ratings_v1.movId.nunique()     # number of movies


            #rating matrix
            rmatrix = dtrain.set_index(['userId','movId']).rating.unstack()
            rmatrix_n=np.array(rmatrix)-np.nanmean(np.matrix(rmatrix),axis=1)
            #change np.matrix to np.array ABOVE
            #ANOTHER CHANGE: add the mean rating per user once(AFTER) the predictions are obtained

            #similarity matrix
            start = time.time()
            sim = np.transpose(pd.DataFrame(rmatrix_n)).corr(method='pearson',
                                                             min_periods=ncm)
            sim.fillna(-1,inplace=True)
            sim=np.array(sim)
            end = time.time()
            print(end-start)

            #Collaborative filtering

            #get neighborhoods
            start=time.time()
            viz=[[np.sort(np.argpartition(sim[u], -nnei)[-nnei:]),
                  sim[u][np.sort(np.argpartition(sim[u], -nnei)[-nnei:])]]

                 for u in range(nu)]
                #print(sim_train[0][viz[0]])

            #exclude non correlated neighbors
            viz=[viz[u][0][np.where(viz[u][1]>min_sim)]

                 for u in range(nu)]

            #exclude the user from its neighborhood
            viz=[np.sort(list(set(viz[u])-set([u]))) 

                 for u in range(nu) ]
                #print(sim_train[0][viz[0]])

            end=time.time()
            print('Neighborhood computation time:\n',end-start)

            #movies to predict
            cf_users=np.where(np.array([len(v) for v in viz])>=min_nei)[0]
            cb_users=list(set(range(nu))-set(cf_users))

            start=time.time()
            mov_pred=[np.sort(list(set(np.where((~np.isnan(rmatrix_n[viz[u],:])).sum(axis=0)!=0)[-1]) -
                                   set(np.where(~np.isnan(rmatrix_n[u,:]))[-1]) 
                                  ))
                      for u in cf_users]
            end=time.time()
            print('\nMovies to predict computation time:\n',end-start)

            #Predict ratings and get top nri recommendations
            start=time.time()
            pred=[]

            rmatrix_n0=np.copy(rmatrix_n)
            rmatrix_n0[np.isnan(rmatrix_n0)] = 0
            for u,v in zip(cf_users, range(len(mov_pred))):

                rmatrix_n_neiu_predm=rmatrix_n0[viz[u]][:,mov_pred[v]]
                sim_u_neiu=sim[u,viz[u]]

                pred_u=np.array(np.matmul(sim_u_neiu,
                                          rmatrix_n_neiu_predm)/np.matmul(np.abs(sim_u_neiu),
                                                                          np.array(~np.isnan(rmatrix_n[viz[u]][:,mov_pred[v]]))
                                                                         )
                               ) #+ np.nanmean(np.array(rmatrix)[u])

                #indexes of movies with greatest ranking from mov_pred[0]
                """

                movie_u=np.argpartition(pred_u, -nri)[-nri:]
                movie_u=mov_pred[v][movie_u]

                pred.append([movie_u,
                             pred_u[np.argpartition(pred_u, -nri)[-nri:]]
                            ])

                Instead of recommending top nri movies, lets see ALL the potencial movies and ratings
                later we can recommend top nri movies
                """
                pred.append(pred_u)

            w=np.array(np.copy(rmatrix))
            predd=[p+np.nanmean(w[u]) for p,u in zip(pred,cf_users)]

            end=time.time()
            print('\nPrediction computation time:\n', end-start)

            ratingsPred=[]
            for ui in range(len(cf_users)):
                u=cf_users[ui]
                for mp,mr in zip(mov_pred[ui],predd[ui]):
                    ratingsPred.append([u,mp,mr])
            ratingsPred_v=pd.DataFrame(ratingsPred,
                                       columns=['userId','movId','ratingPred']).merge(dvald[['userId','movId','rating']],how='inner',on=['userId','movId'])
            print('Normalized RMSE (NRMSE):\n',
                  np.round(
                      100*np.sqrt(np.sum((ratingsPred_v.ratingPred-ratingsPred_v.rating)**2)/ratingsPred_v.shape[0])/5 , 1) )
            print('Normalized MAE (NMAE):\n',
                  np.round(
                      100*np.sqrt(np.sum(np.abs(ratingsPred_v.ratingPred-ratingsPred_v.rating))/ratingsPred_v.shape[0])/5, 1) )
            print('Hit Rate:\n',
                  np.round(ratingsPred_v.shape[0]/dvald.shape[0]*100,1))
            gsearch.append(
                [min_nei,ncm,min_sim,np.round(ratingsPred_v.shape[0]/dvald.shape[0]*100,1) ])
            print([min_nei,ncm,min_sim,np.round(ratingsPred_v.shape[0]/dvald.shape[0]*100,1) ])
            print('\n'*3)
            
            

In [ ]:
"""
Assess evaluation metrics

#for each user:
filtrando através das categorias das recomendações qual a percentagem
das categorias dos filmes ja vistos para esse mesmo user?
E atores?

"""
dtest[dtest.userId.isin(cf_users)].shape[0]


mov_pred[0]
pred[0]

aa=[]
for u,ui in zip(cf_users,range(len(cf_users))):
    
    t3_mi=np.argpartition(pred[ui], -5)[-5:]
    
    t3_mr=pred[ui][t3_mi]
    t3_mo=mov_pred[ui][t3_mi]
    
    for m,mr in zip(t3_mo,t3_mr):
        aa.append([u,m,mr])
aa=pd.DataFrame(aa,columns=['userId','movId','rat_pred']).sort_values(by=['userId','movId'])
aa

c=dtest[dtest.userId.isin(cf_users)].merge(aa,how='left',on=['userId','movId'])
display(c)
display(c[~np.isnan(c.rat_pred)])
#numero users com recomendaçoes vistas/n users nos tests
print('hit rate  (%) : ',c[~np.isnan(c.rat_pred)].userId.nunique()/c.userId.nunique()*100)
print('precision (%) : ',
      100-100*np.mean(np.abs(c[~np.isnan(c.rat_pred)].rating-c[~np.isnan(c.rat_pred)].rat_pred))/(ratings.rating.max()-ratings.rating.min())
     )

In [ ]:
#tagging matrix
tmatrix = tags[~tags.duplicated(subset=['userId','movieId'])]
tmatrix['tag']=1
tmatrix = tmatrix.set_index(['userId','movieId']).tag.unstack()
tmatrix.fillna(0, inplace=True)

addcols = list(set(ratings.movieId.unique()) - set(tmatrix.columns))
addcols_df = pd.DataFrame(np.zeros((tmatrix.shape[0],len(addcols))))
addcols_df.columns = addcols
addcols_df.index = tmatrix.index
tmatrix = pd.concat([tmatrix,addcols_df ], axis=1)

addrows = list(set(ratings.userId.unique()) - set(tmatrix.index))
addrows_df = pd.DataFrame(np.zeros((len(addrows),tmatrix.shape[1])))
addrows_df.columns = tmatrix.columns
addrows_df.index = addrows
tmatrix = pd.concat([tmatrix,addrows_df ], axis=0)

tmatrix.sort_index(axis=0,inplace=True)
tmatrix.sort_index(axis=1,inplace=True)

#genre matrix
gmatrix=movies[['movieId','genres']]
gmatrix=gmatrix.set_index(['movieId']).genres.str.split(pat='|', expand=True).reset_index()
gmatrix=pd.melt(gmatrix, id_vars='movieId',value_vars=gmatrix.columns)
gmatrix=gmatrix[~gmatrix['value'].isin([None])].sort_values(by='movieId')
gmatrix['variable']=1
gmatrix = gmatrix.set_index(['movieId','value']).variable.unstack().fillna(0)
gmatrix = gmatrix[gmatrix.index.isin(np.unique(ratings.movieId))]



aux=pd.DataFrame(ratings.groupby(by='movId').userId.count().sort_values(ascending=True))
display((aux.userId>=16).index)
aux=aux[aux.userId>=16]

ptest=2.5/100
dtest=ratings[ratings.movId.isin(aux.index)].sample(frac=ptest, replace=False,random_state=123)
dtest.sort_index(inplace=True)
dtrain=ratings[~(ratings.index.isin(dtest.index))]
#print(dtrain.userId.nunique(),dtrain.movId.nunique())

####################
print(ratings.groupby('userId').movId.count().describe())

b=pd.DataFrame(dtest.groupby('movId').userId.count().sort_values())
display(b)
c=b.merge(
    ratings.groupby('movId').userId.count().sort_values(),how='left',
    left_on=b.index,right_on=ratings.groupby('movId').userId.count().sort_values().index)
c['dif']=(c.userId_y-c.userId_x)/c.userId_y*100
display(c.sort_values(by='dif',ascending=True))
#display(ratings.groupby('userId').movId.count().head(58))


print(ratings.groupby('movId').userId.count().describe())

b=pd.DataFrame(dtest.groupby('userId').movId.count().sort_values())
display(b)
c=b.merge(
    ratings.groupby('userId').movId.count().sort_values(),how='left',
    left_on=b.index,right_on=ratings.groupby('userId').movId.count().sort_values().index)
c['dif']=(c.movId_y-c.movId_x)/c.movId_y*100
display(c.sort_values(by='dif',ascending=True))



aux=pd.DataFrame(ratings.groupby(by='movId').userId.count().sort_values(ascending=True))
display((aux.userId>=16).index)
aux=aux[aux.userId>=16]

ptest=2.5/100
dtest=ratings[ratings.movId.isin(aux.index)].sample(frac=ptest, replace=False,random_state=123)
dtest.sort_index(inplace=True)
dtrain=ratings[~(ratings.index.isin(dtest.index))]
#print(dtrain.userId.nunique(),dtrain.movId.nunique())

####################
print(ratings.groupby('userId').movId.count().describe())

b=pd.DataFrame(dtest.groupby('movId').userId.count().sort_values())
display(b)
c=b.merge(
    ratings.groupby('movId').userId.count().sort_values(),how='left',
    left_on=b.index,right_on=ratings.groupby('movId').userId.count().sort_values().index)
c['dif']=(c.userId_y-c.userId_x)/c.userId_y*100
display(c.sort_values(by='dif',ascending=True))
#display(ratings.groupby('userId').movId.count().head(58))


print(ratings.groupby('movId').userId.count().describe())

b=pd.DataFrame(dtest.groupby('userId').movId.count().sort_values())
display(b)
c=b.merge(
    ratings.groupby('userId').movId.count().sort_values(),how='left',
    left_on=b.index,right_on=ratings.groupby('userId').movId.count().sort_values().index)
c['dif']=(c.movId_y-c.movId_x)/c.movId_y*100
display(c.sort_values(by='dif',ascending=True))

In [ ]:
#Split Training & Test Set

user_test=np.sort(np.random.choice(range(nu),
                                  int(nu*(1-train_per)),
                                  replace=False)
                )
user_train=np.array(list(set(range(nu))-set(user_test)))

print('Size of test set:',user_test.shape[0],'\n\nUsers:\n',user_test)

sim_train = sim[user_train][:,user_train]
rmatrix_n_train = rmatrix_n[user_train,:]

In [ ]:
#Collaborative filtering

#get neighborhoods
start=time.time()
viz=[[np.sort(np.argpartition(sim_train[u], -nnei)[-nnei:]),
      sim_train[u][np.sort(np.argpartition(sim_train[u], -nnei)[-nnei:])]
     ]
     
     for u in range(user_train.shape[0])]
    #print(sim_train[0][viz[0][0]])

#exclude non correlated neighbors
viz=[viz[u][0][np.where(viz[u][1]>min_sim)]
     
     for u in range(user_train.shape[0])]

#exclude the user from its neighborhood
viz=[np.sort(list(set(viz[u])-set([u]))) 
     
     for u in range(user_train.shape[0]) ]
    #print(sim_train[0][viz[0]])
    
end=time.time()
print('Neighborhood computation time:\n',end-start)

#movies to predict
cf_users=np.where(np.array([len(v) for v in viz])>=min_nei)[0]          #index!
cb_users=np.sort(list(set(user_train)-set(user_train[cf_users])))       #users


start=time.time()
mov_pred=[np.sort(list(set(np.where(np.nansum(rmatrix_n_train[viz[u],:],axis=0)!=0)[-1]) -
                       set(np.where(~np.isnan(rmatrix_n_train[u,:]))[-1]) 
                      ))
          for u in cf_users]
end=time.time()
print('\nMovies to predict computation time:\n',end-start)

#Predict ratings and get top nri recommendations
start=time.time()
pred=[]

rmatrix_n_train0=np.copy(rmatrix_n_train)
rmatrix_n_train0[np.isnan(rmatrix_n_train0)] = 0
for u,v in zip(cf_users, range(len(mov_pred))):
    
    rmatrix_n_neiu_predm=rmatrix_n_train0[viz[u]][:,mov_pred[v]]
    sim_u_neiu=sim_train[u,viz[u]]

    pred_u=np.array(np.matmul(sim_u_neiu,
                              rmatrix_n_neiu_predm)/np.matmul(np.abs(sim_u_neiu),
                                                              rmatrix_n_neiu_predm!=0))

    #indexes of movies with greatest ranking from mov_pred[0]
    movie_u=np.argpartition(pred_u, -nri)[-nri:]
    movie_u=mov_pred[v][movie_u]
    
    pred.append([movie_u,
                 pred_u[np.argpartition(pred_u, -nri)[-nri:]]
                ])
end=time.time()
print('\nPrediction computation time:\n', end-start)

In [ ]:
uid=int(input('Predict recommendation to user Id: '))
while uid in user_test:
    uid=int(input('\nUser Id used to test\nPredict recommendation to user Id: '))

id_train=np.where(user_train==uid)[0][0]
print('id_train:',id_train)
if len(list(viz[id_train])) >=min_nei:
    print('\nNeighbors:\n',list(user_train[viz[id_train]]))

    print('\nPredicted movies:\n')
    id_new=np.where(user_train[cf_users]==uid)[0][0]
    print('id_new: ',id_new)
    df=pd.DataFrame(np.transpose(pred[id_new]),
                   columns=['mov_id','prediction'])
    df.mov_id=df.mov_id.astype('int64')
    df.sort_values(by='prediction',ascending=False,inplace=True)
    df.index=[1,2,3,4,5]
    display(df.merge(movies,on='mov_id'))

    print('\n'*2,'*'*20,'\n')
    display(ratings[ratings.userId==uid+1].merge(movies,
                                                 left_on='movieId', right_on='movieId').head(10))
else:
    print('\nsmall neighborhood: ',len(list(viz[id_train])),'\nNo prediction')

In [ ]:
#Test set - Error

#get neighborhoods
start=time.time()
vizt=[np.sort(np.argpartition(sim[u], -nnei)[-nnei:])
      for u in user_test]
end=time.time()
print('Neighborhood computation time:\n',end-start)
    #print(vizt[0])

#exclude the user from its neighborhood
vizt=[np.sort(list(set(vizt[v])-set([u])))
      for u,v in zip(user_test,range(user_test.shape[0])) ]

    #print(vizt[0])

#movies to predict
start=time.time()
mov_pred=[np.where(~np.isnan(rmatrix_n[u]))[0]
          for u in user_test]
end=time.time()
print('\nMovies to predict computation time:\n',end-start)
    #print(mov_pred[0])

    #display(ratings[ratings.userId.isin(user_test+1)].sort_values(['userId','movieId']).head(30))

#Predict ratings and get top nri recommendations
start=time.time()
rated=np.sort(ratings.movieId.unique())
pred=[]

rmatrix_n_t=np.copy(rmatrix_n)
rmatrix_n_t[np.isnan(rmatrix_n_t)] = 0
for u,v in zip(user_test,range(len(vizt)) ):
    
    rmatrix_n_neiu_predm=rmatrix_n_t[vizt[v]][:,mov_pred[v]]
    sim_u_neiu=sim[u,vizt[v]]

    pred_u=np.array(np.matmul(sim_u_neiu,
                              rmatrix_n_neiu_predm)/np.matmul(np.abs(sim_u_neiu),
                                                              ~np.isnan(rmatrix_n[vizt[v]][:,mov_pred[v]]))
                   )
    
    pred.append(pred_u)
end=time.time()
print('\nPrediction computation time:\n', end-start)

print('\nMSE:')
mse_n=0
mse=0
for u,v in zip(user_test,range(len(pred))):
    
    error=(rmatrix_n[u][:,mov_pred[v]]-pred[v])
    error= np.array(error)**2
    error=error[~np.isnan(error)]
    
    mse_n=mse_n+error.shape[0]
    mse=mse+np.sum(error)
print(mse/mse_n)

In [ ]:
#rascunho

#np.where(rmatrix.columns==2)[0].tolist()[0]
mov_pred[0]
#rmatrix_n[0,:][:,np.where(rmatrix_n[0,:]!=0)[1]]
#rmatrix_n[0,:]

#predict ratings

#def prev_rat(neigh,ratings):
np.matrix(rmatrix)[:,mov_pred[0]][j[0],]
a=np.matrix([[2,np.nan,3,np.nan],
             [np.nan,3,3,np.nan],
             [5,4,np.nan,4],
             [np.nan,2,5,np.nan]])
#print(a)
a[np.isnan(a)] = 0
print(a)
print(np.matmul(np.matrix([-1,1,0.5,1]),a))

print(a!=0)
print(np.matmul(np.matrix([-1,1,0.5,1]),a!=0))

print('\n'*2)
print(np.matmul(np.matrix([-1,1,0.5,1]),a)/np.matmul(np.matrix([-1,1,0.5,1]),a!=0))

print("\n"*2)
print("~"*20)
print("\n"*3)

#verficar que os filmes/items selecionados sao realmente os items e nao as posiçoes 

rmatrix_u1_items_norm=rmatrix_n[j[0]][:,mov_pred[0]]
rmatrix_u1_items_norm[np.isnan(rmatrix_u1_items_norm)]=0
sim_u1=np.matrix(sim)[0,j[0]]

"""
Movies Prediction list

It is missing
+np.nanmean(np.matrix(rmatrix)[0]))

"""
k=np.array(np.matmul(sim_u1,
                     rmatrix_u1_items_norm)/np.matmul(np.abs(sim_u1),
                                                      rmatrix_u1_items_norm!=0))
k=k[0]

print(np.max(k))
print(np.where(k==np.max(k))[0])
print(k[np.where(k==np.max(k))[0]])

print('holla')
print(k.argsort()[-5:])
print(k[k.argsort()[-5:]])

#movie_u1=np.argmax(mov_pred[0])

movie_u1=k.argsort()[-5:]
#+ np.nanmean(np.matrix(rmatrix)[0]))]
#print(movie_u1)
#print('ola')
#print(k[movie_u1])
#rmatrix.columns[movie_u1]

rated=np.sort(ratings.movieId.unique())
#print(rated)
rated[k.argsort()[-5:]]

print(np.where(np.array([len(v) for v in viz])>=min_nei)[0],'\n'*2)
for u in np.where(np.array([len(v) for v in viz])>=min_nei)[0]:
    print(viz[u])

In [ ]:
#verificar que para o primeiro user o item a recomendar é o item 895
a1=np.sort(np.argsort(np.matrix(sim.fillna(-1))[0])[0,-10:])
a1=np.sort(list(set(a1.tolist()[0])-set([0])))
print(a1)
print('\n'*3)

movies_to_pred_u1 =np.where(np.sum(np.matrix(rmatrix.fillna(0))[j[0],:],axis=0)>0)[1]
print(movies_to_pred_u1)

In [ ]:
#EDA

display(ratings.head())
print('Num unique users: ' ,ratings.userId.nunique() )
print('Num unique movies: ',ratings.movieId.nunique())
print('\n')
print(ratings.rating.describe())

ratings.groupby(by='userId').movieId.agg('nunique').describe()
dfr=pd.DataFrame(ratings.groupby(by='movieId').rating.agg('mean'))
dfr[(dfr.rating >=4) & (dfr.rating<=5)].shape
dfr1=pd.DataFrame(ratings.groupby(by='movieId').userId.agg('nunique'))
dfr1[dfr1.userId>=5].shape

In [ ]:
#content based RS
"""
user_cat_matrix = [
    np.multiply(
        np.tile(np.matrix(rmatrix.fillna(0))[i,:].T, (1, 20)),
        np.matrix(gmatrix)
    )
    for i in range(610)
]

user_cat_vect = [
    np.sum(user_cat,axis=0)/np.sum(np.sum(user_cat,axis=0))
    for user_cat in user_cat_matrix
]

pred_rat=[
    np.sum(
        np.multiply(
            np.tile(user_cat_vect[u], (9724, 1)) ,
            np.matrix(gmatrix)
        ),
        axis=1)
    for u in range(610)
]
"""

#faster
x1=np.matmul(np.matrix(rmatrix.fillna(0)),
          np.matrix(gmatrix)
         )
x1=normalize(x1,axis=1,norm='l1')

x2=np.matmul(x1,np.matrix(gmatrix).T)

In [ ]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(n_neighbors=25,
                         metric='cosine',
                         algorithm='brute',
                         n_jobs=-1)
model.fit(rmatrix)


In [ ]:
u=0
np.sort(list(set(np.where(np.sum(rmatrix_n_train[viz[u],:],axis=0)!=0)[1]) -
                       set(np.where(rmatrix_n_train[u,:]!=0)[1]) 
                      ))
#Collaborative filtering

#get neighborhoods
start=time.time()
viz=[np.sort(np.argpartition(sim_train[u], -nnei)[-nnei:])
     for u in range(user_train.shape[0])]
end=time.time()

print('Neighborhood computation time:\n',end-start)

#exclude the user from its neighborhood
viz=[np.sort(list(set(viz[u])-set([u]))) 
     for u in range(user_train.shape[0]) ]

#movies to predict
start=time.time()
mov_pred=[np.sort(list(set(np.where(np.sum(rmatrix_n_train[viz[u],:],axis=0)!=0)[1]) -
                       set(np.where(rmatrix_n_train[u,:]!=0)[1]) 
                      ))
          for u in range(user_train.shape[0])]
end=time.time()
print('\nMovies to predict computation time:\n',end-start)

#Predict ratings and get top nri recommendations
start=time.time()
rated=np.sort(ratings.movieId.unique())
pred=[]

for u in range(user_train.shape[0]):
    
    rmatrix_n_neiu_predm=rmatrix_n_train[viz[u]][:,mov_pred[u]]
    sim_u_neiu=sim_train[u,viz[u]]

    pred_u=np.array(np.matmul(sim_u_neiu,
                              rmatrix_n_neiu_predm)/np.matmul(np.abs(sim_u_neiu),
                                                              rmatrix_n_neiu_predm!=0))
    
    pred_u=pred_u[0]
    #indexes of movies with greatest ranking from mov_pred[0]
    movie_u=np.argpartition(pred_u, -nri)[-nri:]
    movie_u=mov_pred[u][movie_u]

    pred.append([rated[movie_u],
                 pred_u[np.argpartition(pred_u, -5)[-5:]]
                ])
end=time.time()
print('\nPrediction computation time:\n', end-start)